In [4]:
import numpy as np
import BayesPower as BP
import pickle, json, os, sys
from enterprise_extensions.model_utils import get_tspan

# pip install fastshermanmorrison-pulsar

fbins = 5
pidx = 0

datadir = '/home/koonima/FAST/Data/Pickle/'
with open(datadir + f'v1p1_de440_pint_bipm2019.pkl', 'rb') as fin:
    psrs = pickle.load(fin)[pidx]

m = BP.BayesPowerSingle(psr = psrs,
            Tspan = get_tspan([psrs]),
            select = 'backend',
            white_vary = True,
            inc_ecorr = True,
            tnequad=False,
            ecorr_type = 'kernel',
            noise_dict = None,
            tm_marg = False,
            freq_bins = fbins,
            log10rhomin=-9.0,
            log10rhomax=-4.5,)

libstempo not installed. Will use PINT instead.


In [5]:
%load_ext line_profiler

In [6]:
%lprun -f m.sample m.sample(niter=int(10), resume=False, progress_bar=True, savepath = f'./Chain/{fbins}_FS_SP')

Sampling B1855+09: 100%|██████████| 10/10 [00:12<00:00,  1.21s/it]


Timer unit: 1e-09 s

Total time: 12.1427 s
File: /home/koonima/pandora/pandora/NG20/SP/BayesPower.py
Function: sample at line 429

Line #      Hits         Time  Per Hit   % Time  Line Contents
   429                                               def sample(
   430                                                   self,
   431                                                   niter=int(1e4),
   432                                                   resume = True,
   433                                                   wniters=1,
   434                                                   eciters=1,
   435                                                   savepath=None,
   436                                                   SCAMweight=30,
   437                                                   AMweight=15,
   438                                                   DEweight=50,
   439                                                   covUpdate=1000,
   440                                  

In [9]:
m.pta

<Enterprise PTA object: B1855+09>

In [10]:
x0 = np.hstack([p.sample() for p in m.pta.params])
x0

array([ 9.75821371, -7.95187792, -5.72868744,  1.30130464, -5.77839395,
       -7.50986573,  2.52075166, -5.06393997, -5.52046304,  5.28766335,
       -8.21763109, -6.69494205, -7.43851783, -8.56279413, -7.76131767,
       -6.93792844, -8.02752623])

In [11]:
ndim = len(x0)
cov = np.diag(np.ones(ndim) * 0.01**2)
groups = [list(np.arange(0, ndim))]

In [13]:
from PTMCMCSampler.PTMCMCSampler import PTSampler as ptmcmc
from enterprise.signals import signal_base, gp_signals
from scipy.linalg import cho_factor, cho_solve
from enterprise_extensions import sampler as samp

In [20]:
sampler = ptmcmc(
    ndim,
    m.pta.get_lnlikelihood,
    m.pta.get_lnprior,
    cov,
    groups=groups,
    outDir='./TestChain',
    resume=False,
)

In [17]:
def draw_from_prior(x, iter, beta):
    """Prior draw.

    The function signature is specific to PTMCMCSampler.
    """

    q = x.copy()
    lqxy = 0

    # randomly choose parameter
    param_idx = random.randint(0, x.shape[0] - 1)
    q[param_idx] = np.random.uniform(-8., -2, size = crn_bins)[param_idx]
    return q, float(lqxy)

In [21]:
# sampler.addProposalToCycle(draw_from_prior, 10)

In [22]:
sampler.sample(
    x0,
    int(5e5),
    SCAMweight=30,
    AMweight=15,
    DEweight=50,
)

Finished 0.00 percent in 0.000235 s Acceptance rate = 0

/home/koonima/anaconda3/envs/pandora/lib/python3.11/site-packages/enterprise/signals/parameter.py:70: RuntimeWarning: divide by zero encountered in log
  logpdf = np.log(self.prior(value, **kwargs))


Finished 0.20 percent in 102.553350 s Acceptance rate = 0.731

KeyboardInterrupt: 